## Notebook for generating 'pretrained' features for CE (SL)

**Created on 30th October; modified on 10th November, 2022**

In [1]:
import warnings
warnings.filterwarnings('ignore')
import deepchem
# import chemprop
import rdkit
import sklearn
import mordred
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import torch.nn as nn
from sklearn.decomposition import PCA
import numpy as np
from transformers import AutoModelForMaskedLM, AutoTokenizer, pipeline, RobertaModel, RobertaTokenizer
from tqdm import tqdm
from rdkit import Chem
# from ifg import identify_functional_groups ## Need ifg.py file
import rdkit.Chem.rdMolDescriptors as MolDescriptors
import rdkit.Chem.Descriptors as Descriptors
import rdkit.Chem.Lipinski as Lipinski
from rdkit.Chem import Descriptors3D
from rdkit.Chem import AllChem, DataStructs
from mordred import Calculator, descriptors, CPSA
from mordred.GeometricalIndex import Radius3D, Diameter3D, GeometricalShapeIndex

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


In [3]:
%%bash
pwd
ls -ltr

/Users/ritesh2/Library/CloudStorage/Box-Box/Research-postdoc/liquid-electrolyte-ML/coulombic-efficiency/11-09-22/random-split/with-10-features-each
total 960
-rw-------@ 1 ritesh2  staff   79036 Nov 10 23:21 all_CE_ED_pre_feat.csv
-rw-------@ 1 ritesh2  staff    8137 Nov 10 23:21 CE_ood_pre_feat.csv
-rw-------@ 1 ritesh2  staff   79036 Nov 10 23:22 tot_CE_ED_ood_pre_feat.csv
-rw-------@ 1 ritesh2  staff  318433 Nov 10 23:23 gen_pretrained_feat_pca.ipynb


In [3]:
# df_ea = pd.read_csv('preds_ce_edb_chemprop_random.csv')
# df_ea

,smiles,IE,IE_ensemble_uncal_var,EA,EA_ensemble_uncal_var
0,CC1COC(=O)O1,6.832252,0.047126,-0.767132,0.029125
1,C1CCOC1,5.468410,0.015913,-1.009737,0.014155
2,CC1CCCO1,5.503122,0.021355,-0.922206,0.023058
3,CCOCC,5.521860,0.021856,-1.221115,0.035728
4,COCCOC,5.559792,0.022901,-0.693981,0.100682
...,...,...,...,...,...
61,COCCOC(C)C,5.462625,0.019827,-1.039499,0.020157
62,CCOCCOC(C)C,5.397021,0.014543,-1.174437,0.029217
63,CCC[Si](OC)(OC)OC,5.635698,0.013896,-0.821758,0.018594
64,CCCCCC[Si](OC)(OC)OC,5.546748,0.025857,-0.881437,0.016781


In [4]:
# df_ea['EA'].min()

-1.2536746976612252

In [44]:
df = pd.read_csv('tot_CE_ED_ood_pre_feat.csv')
df

,solvent_1_smiles,solvent_2_smiles,solvent_3_smiles,salt_1_smiles,salt_2_smiles,additive_smiles,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,...,pre_feat_add_12,pre_feat_add_13,pre_feat_add_14,pre_feat_add_15,num_heavy_atoms_add,mw_add,protocol,current_density,coulombic_efficiency,log(1-CE)
0,CC1COC(=O)O1,NaN,NaN,[Li+].[O-][Cl+3]([O-])([O-])[O-],NaN,NaN,17.408940,-39.545807,-22.604496,17.312720,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,0,2.0,0.800,-1.609438
1,CC1COC(=O)O1,NaN,NaN,[Li+].[O-][Cl+3]([O-])([O-])[O-],NaN,O=S=O,17.408472,-39.545840,-22.604517,17.312712,...,3.170000e-09,-4.740000e-11,-1.970000e-09,-7.250000e-10,3,63.9619,1,5.0,0.836,-1.807889
2,C1CCOC1,NaN,NaN,F[As-](F)(F)(F)(F)F.[Li+],NaN,NaN,64.222480,-1.441071,-4.633460,17.839956,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,1,5.0,0.894,-2.244316
3,CC1CCCO1,NaN,NaN,F[As-](F)(F)(F)(F)F.[Li+],NaN,NaN,248.554760,27.599460,-1.178202,16.094427,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,1,0.9,0.974,-3.649659
4,CC1CCCO1,NaN,NaN,F[As-](F)(F)(F)(F)F.[Li+],NaN,NaN,248.554760,27.599460,-1.178156,16.094429,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,1,0.9,0.974,-3.649659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,COCCOC(C)C,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,NaN,257.665560,-92.078210,6.875039,81.282670,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.985,-4.199705
152,CCOCCOC(C)C,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,NaN,77.102196,-93.227230,2.249297,64.492090,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.989,-4.509860
153,CCC[Si](OC)(OC)OC,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,NaN,-106.717125,0.973197,-2.709262,-3.330054,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.980,-3.912023
154,CCCCCC[Si](OC)(OC)OC,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,NaN,-477.186700,-25.545970,-14.940089,4.034881,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.857,-1.944911


Combine smiles

In [50]:
ea_solv_1 = []
ea_solv_2 = []
ea_solv_3 = []
solv_comb = []
salt_comb = []
for i in range(len(df)):
    if df['solv_ratio_2'][i] == 0.0:
        solv_1 = df['solvent_1_smiles'][i]
        print(solv_1)
        solv_comb.append(solv_1)
        # ea_solv_1.append(float(df_ea.loc[df_ea['smiles'] == solv_1]['EA']))
        # ea_solv_2.append(0.0)
        # ea_solv_3.append(0.0)
    elif df['solv_ratio_2'][i] != 0.0 and df['solv_ratio_3'][i] == 0.0:
        solv_1 = df['solvent_1_smiles'][i]
        solv_2 = df['solvent_2_smiles'][i]
        smile_comb = df['solvent_1_smiles'][i] + '.' + df['solvent_2_smiles'][i]
        solv_comb.append(smile_comb)
        # ea_solv_1.append(float(df_ea.loc[df_ea['smiles'] == solv_1]['EA']))
        # ea_solv_2.append(float(df_ea.loc[df_ea['smiles'] == solv_2]['EA']))
        # ea_solv_3.append(0.0)
    else:
        solv_1 = df['solvent_1_smiles'][i]
        solv_2 = df['solvent_2_smiles'][i]
        solv_3 = df['solvent_3_smiles'][i]
        smile_comb = df['solvent_1_smiles'][i] + '.' + df['solvent_2_smiles'][i] + '.' + df['solvent_3_smiles'][i]
        solv_comb.append(smile_comb)
        # ea_solv_1.append(float(df_ea.loc[df_ea['smiles'] == solv_1]['EA']))
        # ea_solv_2.append(float(df_ea.loc[df_ea['smiles'] == solv_2]['EA']))
        # ea_solv_3.append(float(df_ea.loc[df_ea['smiles'] == solv_3]['EA']))

CC1COC(=O)O1
CC1COC(=O)O1
C1CCOC1
CC1CCCO1
CC1CCCO1
COCCOC
O=C1OCC(F)O1
COCCOC
CN(C)S(=O)(=O)F
COCC(F)(F)C(F)(F)COC
CCN(CC)S(=O)(=O)C(F)(F)F
C1COCO1
CC1COC(=O)O1
C1CCOC1
CC1CCCO1
COCOC
O=c1occo1
O=C1OCC(F)O1
COCCOC
COCCOC
COCCOC
COCCOC
COCCOCCOCC(F)(F)OC(F)(F)C(F)(F)OC(F)(F)COCCOCCOC
COCCCCOC
COCCOC
COCCOC
COCCOC
COC(=O)OC
O=C1OCC(F)O1
CCOC(=O)OCC
COCCOCCOC
COCCOCCOCCOCCOC
COCCOC
COCCOCCOC
COCCOCCOCCOC
COCCOCCOCCOCCOC
COCCOCCOCCOCCOC
COC(=O)OC
COCC(F)(F)C(F)(F)COC
CCOCCOCC
CCOCCOCC(F)(F)F
FC(F)(F)COCCOCC(F)(F)F
FC(F)COCCOCC(F)F
FC(F)COCCOCC(F)(F)F
COCC(F)(F)C(F)(F)COC
COCCOC
COC(=O)OC
CCn1cc[n+](C)c1.O=S(=O)(F)[N-]S(=O)(=O)F
CO[Si](C)(C)OC
CCCCOCCCC
COC(=O)OCC(F)(F)F
COC(=O)OCC(F)(F)F
C1COCCO1
CCO[Si](OCC)(OCC)OCC
CCO[Si](OC)(OCC)OCC
CCO[Si](C)(C)OCC
CO[Si](C)(C)OC
O=S1(=O)CCCC1
COC1OCC(C(F)(F)F)O1
COC1(OC)OCC(C(F)(F)F)O1
FC(F)(F)COCCOCCOCC(F)(F)F
FCCOCCOCCOCCF
FCCOB(OCCF)OCCF
CCOC(OCC)OCC
COCCOCC(C)C
COCCOB(OCCOC)OCCOC
FCCOB(OCCF)OCCF
COCCOC(C)(C)C
CCCCOCCOC
COCCOC(C)C
CCOCCOC(C)C
CCC

In [51]:
solv_comb

['CC1COC(=O)O1',
 'CC1COC(=O)O1',
 'C1CCOC1',
 'CC1CCCO1',
 'CC1CCCO1',
 'CCOCC.C1CCOC1',
 'CC1CCCO1.COCCOC',
 'O=C1OCCO1.C1COCO1',
 'C1COCO1.COCOC',
 'COCCOC',
 'O=C1OCC(F)O1',
 'C1CCOC1.CF.O=C=O',
 'O=C1OCCO1.COC(=O)OC.CCOC(=O)OCC',
 'O=C1OCC(F)O1.FC(F)(F)COC1OCO1.FC(F)C(F)(F)OCC(F)(F)F',
 'COCCOC',
 'O=C1OCCO1.COC(=O)OC',
 'COC(=O)OC.FC(F)(F)COCC(F)(F)F',
 'COCCOC.FC(F)(F)COC(OCC(F)(F)F)OCC(F)(F)F',
 'C1COCO1.COCCOC',
 'CN(C)S(=O)(=O)F',
 'CC#N.CF',
 'COCC(F)(F)C(F)(F)COC',
 'CCN(CC)S(=O)(=O)C(F)(F)F',
 'O=C1OCCO1.CCOC(=O)OCC',
 'C1COCO1',
 'O=C1OCCO1.CCOC(=O)OC',
 'O=C1OCCO1.CCOC(=O)OCC',
 'C1COCO1.COCOC',
 'CC1COC(=O)O1',
 'C1CCOC1',
 'CC1CCCO1.COCCCOC',
 'CC1CCCO1',
 'O=C1OCCO1.CC1COC(=O)O1',
 'COCOC',
 'O=C1OCCO1.COC(=O)OC',
 'O=C1OCCO1.CCOC(=O)OC',
 'O=C1OCCO1.CCOC(=O)OCC',
 'O=C1OCCO1.CCOC(=O)OC',
 'O=C1OCCO1.CCOC(=O)OC',
 'O=C1OCCO1.COC(=O)OC',
 'O=C1OCCO1.CCOC(=O)OCC',
 'O=C1OCCO1.CCOC(=O)OCC',
 'O=C1OCCO1.CCOC(=O)OCC',
 'O=C1OCCO1.COC(=O)OC',
 'O=c1occo1',
 'O=C1OCCO1.CCOC(

In [52]:
for i in range(len(df)):
    if df['salt_2_conc'][i] == 0.0:
        salt_comb.append(df['salt_1_smiles'][i])
    else:
        smile = df['salt_1_smiles'][i] + '.' + df['salt_2_smiles'][i]
        salt_comb.append(smile)
salt_comb

['[Li+].[O-][Cl+3]([O-])([O-])[O-]',
 '[Li+].[O-][Cl+3]([O-])([O-])[O-]',
 'F[As-](F)(F)(F)(F)F.[Li+]',
 'F[As-](F)(F)(F)(F)F.[Li+]',
 'F[As-](F)(F)(F)(F)F.[Li+]',
 'F[As-](F)(F)(F)(F)F.[Li+]',
 'F[As-](F)(F)(F)(F)F.[Li+]',
 'F[As-](F)(F)(F)(F)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+].O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F.[Li+]',
 'F[P-](F)(F)(F)(F)F.[Li+]',
 'F[P-](F)(F)(F)(F)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+].O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)([N-]S(=O)(=O)C(F)(F)F)C(F)(F)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 'O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]',
 '[Li+].[O-][Cl+3]([O-])([O-])[O-]',
 '[Li+].[O-][Cl+3]([O-])([O-])[O-]',
 'O=C1O[B-](F)(F)OC1=O.[Li+]',
 'O=C1O[B-

In [53]:
df['solv_comb_sm'] = solv_comb
df['salt_comb_sm'] = salt_comb

**Generate solvent features**

In [54]:
model = AutoModelForMaskedLM.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")
tokenizer = AutoTokenizer.from_pretrained("seyonec/PubChem10M_SMILES_BPE_450k")

# fill_mask = pipeline('fill-mask', model=model, tokenizer=tokenizer)
for param in model.parameters():
    param.requires_grad = False

In [55]:
solv_tens_feat = df["solv_comb_sm"].map(lambda smiles: model(**tokenizer(smiles, return_tensors="pt", padding=True))["logits"][0, 0, :])
solv_tens_feat[0].shape

torch.Size([52000])

In [56]:
solv_tens_feat_stack = torch.stack(solv_tens_feat.tolist()).numpy()
solv_tens_feat_stack.shape

(156, 52000)

In [57]:
solv_tens_feat_stack

array([[ -7.790232 , -12.475418 ,  -7.572564 , ..., -12.817182 ,
        -12.942478 , -12.794449 ],
       [ -7.790232 , -12.475418 ,  -7.572564 , ..., -12.817182 ,
        -12.942478 , -12.794449 ],
       [ -7.7043366, -12.283732 ,  -8.584637 , ..., -12.785764 ,
        -12.510005 , -12.443646 ],
       ...,
       [ -7.297457 , -12.906971 ,  -6.7649508, ..., -13.954237 ,
        -13.758419 , -13.360327 ],
       [ -7.514679 , -14.635654 ,  -6.254439 , ..., -15.74994  ,
        -15.474209 , -15.361111 ],
       [ -7.5979695, -11.358162 ,  -7.3955464, ..., -12.035589 ,
        -11.665693 , -11.833992 ]], dtype=float32)

In [58]:
nPC = 10
pca = PCA(nPC)
solv_feat_transf = pca.fit_transform(solv_tens_feat_stack)
solv_feat_transf.shape

(156, 10)

In [59]:
print(solv_feat_transf[0])
print(solv_feat_transf[1])
print(solv_feat_transf[141])

[ 13.960964  -36.666103  -22.260607   16.980927    8.733797   24.34312
 -13.19099   -21.466457   -0.9576501 -12.557255 ]
[ 13.96078   -36.666084  -22.260595   16.980879    8.7338295  24.343128
 -13.190999  -21.466469   -0.9577008 -12.557279 ]
[-4.8488464e+02 -5.5837299e+01  1.3064472e+01 -2.1287367e+01
  9.8423939e+00 -1.5781491e-01  4.9980049e+00  8.4974051e+00
 -1.2208233e+01  6.6278086e+00]


In [60]:
solv_feat_transf.T[0]

array([ 1.39609642e+01,  1.39607801e+01,  6.08116226e+01,  2.45147781e+02,
        2.45147781e+02, -1.54553284e+02,  3.04110199e+02, -1.12029282e+02,
        1.81751785e+02, -5.29239441e+02,  7.25177124e+02, -1.66333496e+02,
       -1.71272568e+02,  2.56881165e+02, -5.29239441e+02,  5.02301931e+00,
       -5.36340088e+02, -1.76580902e+02,  3.69466034e+02,  5.67124939e+02,
        1.07714648e+03, -2.37993271e+02, -1.87407394e+02,  2.64508148e+02,
        1.00626846e+02,  4.67203178e+01,  2.64508148e+02,  1.81751785e+02,
        1.39607801e+01,  6.08116226e+01,  2.15430283e+02,  2.45147781e+02,
       -3.85472137e+02, -4.71407135e+02,  5.02301931e+00,  4.67203178e+01,
        2.64508148e+02,  4.67203178e+01,  4.67203178e+01,  5.02301931e+00,
        2.64508148e+02,  2.64508148e+02,  2.64508148e+02,  5.02301931e+00,
        4.50443146e+02,  4.67203178e+01,  6.30527830e+00,  2.64508148e+02,
        7.25177124e+02,  4.67203178e+01,  2.64508148e+02,  8.75907059e+01,
       -6.38775368e+01,  

In [61]:
solv_dict = {}
for i in range(10):
    feat_name = 'pre_feat_solv_' + str(i+1)
    solv_dict[feat_name] = solv_feat_transf.T[i]

In [62]:
solv_dict

{'pre_feat_solv_1': array([ 1.39609642e+01,  1.39607801e+01,  6.08116226e+01,  2.45147781e+02,
         2.45147781e+02, -1.54553284e+02,  3.04110199e+02, -1.12029282e+02,
         1.81751785e+02, -5.29239441e+02,  7.25177124e+02, -1.66333496e+02,
        -1.71272568e+02,  2.56881165e+02, -5.29239441e+02,  5.02301931e+00,
        -5.36340088e+02, -1.76580902e+02,  3.69466034e+02,  5.67124939e+02,
         1.07714648e+03, -2.37993271e+02, -1.87407394e+02,  2.64508148e+02,
         1.00626846e+02,  4.67203178e+01,  2.64508148e+02,  1.81751785e+02,
         1.39607801e+01,  6.08116226e+01,  2.15430283e+02,  2.45147781e+02,
        -3.85472137e+02, -4.71407135e+02,  5.02301931e+00,  4.67203178e+01,
         2.64508148e+02,  4.67203178e+01,  4.67203178e+01,  5.02301931e+00,
         2.64508148e+02,  2.64508148e+02,  2.64508148e+02,  5.02301931e+00,
         4.50443146e+02,  4.67203178e+01,  6.30527830e+00,  2.64508148e+02,
         7.25177124e+02,  4.67203178e+01,  2.64508148e+02,  8.7590705

In [63]:
solv_df = pd.DataFrame(solv_dict)
solv_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10
0,13.960964,-36.666103,-22.260607,16.980927,8.733797,24.343121,-13.190990,-21.466457,-0.957650,-12.557255
1,13.960780,-36.666084,-22.260595,16.980879,8.733829,24.343128,-13.190999,-21.466469,-0.957701,-12.557279
2,60.811623,0.946545,-4.255921,20.332489,45.908714,-7.113601,-13.550858,-3.377986,6.305635,-2.023123
3,245.147781,29.954426,-0.577181,18.008678,45.957737,14.886771,-29.603376,10.542392,11.490660,12.857011
4,245.147781,29.954430,-0.577294,18.008636,45.957726,14.886753,-29.603340,10.542382,11.490674,12.857035
...,...,...,...,...,...,...,...,...,...,...
151,254.200806,-89.463516,6.972825,83.248619,9.417171,14.582830,22.372253,14.008273,10.369975,-12.614752
152,73.645363,-90.855064,2.386254,67.773544,22.316572,6.024801,30.230356,8.795265,20.386084,-16.042814
153,-110.040062,2.599045,-3.462972,-0.363976,3.405136,-12.481925,43.612568,-2.068670,-13.263947,26.234848
154,-480.540283,-23.481226,-15.529446,6.733684,3.938678,-16.251133,38.950779,-8.679785,-8.750237,17.551256


In [64]:
num_atom_solv_1 = []
num_atom_solv_2 = []
num_atom_solv_3 = []
mw_solv_1 = []
mw_solv_2 = []
mw_solv_3 = []
for i in range(len(df)):
    if df['solv_ratio_2'][i] == 0.0:
        mol1 = Chem.MolFromSmiles(df['solvent_1_smiles'][i])
        num_atom_solv_1.append(MolDescriptors.CalcNumHeavyAtoms(mol1))
        mw_solv_1.append(MolDescriptors.CalcExactMolWt(mol1))
        num_atom_solv_2.append(0.0)
        num_atom_solv_3.append(0.0)
        mw_solv_2.append(0.0)
        mw_solv_3.append(0.0)
    elif df['solv_ratio_2'][i] != 0.0 and df['solv_ratio_3'][i] == 0.0:
        mol1 = Chem.MolFromSmiles(df['solvent_1_smiles'][i])
        mol2 = Chem.MolFromSmiles(df['solvent_2_smiles'][i])
        num_atom_solv_1.append(MolDescriptors.CalcNumHeavyAtoms(mol1))
        num_atom_solv_2.append(MolDescriptors.CalcNumHeavyAtoms(mol2))
        mw_solv_1.append(MolDescriptors.CalcExactMolWt(mol1))
        mw_solv_2.append(MolDescriptors.CalcExactMolWt(mol2))
        num_atom_solv_3.append(0.0)
        mw_solv_3.append(0.0)
    else:
        mol1 = Chem.MolFromSmiles(df['solvent_1_smiles'][i])
        mol2 = Chem.MolFromSmiles(df['solvent_2_smiles'][i])
        mol3 = Chem.MolFromSmiles(df['solvent_3_smiles'][i])
        num_atom_solv_1.append(MolDescriptors.CalcNumHeavyAtoms(mol1))
        num_atom_solv_2.append(MolDescriptors.CalcNumHeavyAtoms(mol2))
        num_atom_solv_3.append(MolDescriptors.CalcNumHeavyAtoms(mol3))
        mw_solv_1.append(MolDescriptors.CalcExactMolWt(mol1))
        mw_solv_2.append(MolDescriptors.CalcExactMolWt(mol2))
        mw_solv_3.append(MolDescriptors.CalcExactMolWt(mol3))

In [65]:
num_atom_salt_1 = []
num_atom_salt_2 = []
mw_salt_1 = []
mw_salt_2 = []
for i in range(len(df)):
    if df['salt_2_conc'][i] == 0.0:
        mol1 = Chem.MolFromSmiles(df['salt_1_smiles'][i])
        num_atom_salt_1.append(MolDescriptors.CalcNumHeavyAtoms(mol1))
        mw_salt_1.append(MolDescriptors.CalcExactMolWt(mol1))
        num_atom_salt_2.append(0.0)
        mw_salt_2.append(0.0)
    else:
        mol1 = Chem.MolFromSmiles(df['salt_1_smiles'][i])
        mol2 = Chem.MolFromSmiles(df['salt_2_smiles'][i])
        num_atom_salt_1.append(MolDescriptors.CalcNumHeavyAtoms(mol1))
        num_atom_salt_2.append(MolDescriptors.CalcNumHeavyAtoms(mol2))
        mw_salt_1.append(MolDescriptors.CalcExactMolWt(mol1))
        mw_salt_2.append(MolDescriptors.CalcExactMolWt(mol2))
mw_salt_2

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 286.93329835,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 286.93329835,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 286.93329835,
 286.93329835,
 286.93329835,
 286.93329835,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 286.93329835,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 286.93329835,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 94.01892283000001,
 105.96451571,
 144.00177487,
 155.96802907,
 151.9801855,
 286.93329835,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,


In [66]:
num_atom_salt_2

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 16,
 16,
 16,
 16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 6,
 6,
 10,
 9,
 8,
 16,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [67]:
# solv_df['solv_comb_sm'] = solv_comb
solv_df['solv_ratio_1'] = df['solv_ratio_1']
solv_df['solv_ratio_2'] = df['solv_ratio_2']
solv_df['solv_ratio_3'] = df['solv_ratio_3']
solv_df['num_heavy_atoms_solv_1'] = num_atom_solv_1
solv_df['num_heavy_atoms_solv_2'] = num_atom_solv_2
solv_df['num_heavy_atoms_solv_3'] = num_atom_solv_3
solv_df['mw_solv_1'] = mw_solv_1
solv_df['mw_solv_2'] = mw_solv_2
solv_df['mw_solv_3'] = mw_solv_3
# solv_df['ea_solv_1'] = ea_solv_1
# solv_df['ea_solv_2'] = ea_solv_2
# solv_df['ea_solv_3'] = ea_solv_3
solv_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10,solv_ratio_1,solv_ratio_2,solv_ratio_3,num_heavy_atoms_solv_1,num_heavy_atoms_solv_2,num_heavy_atoms_solv_3,mw_solv_1,mw_solv_2,mw_solv_3
0,13.960964,-36.666103,-22.260607,16.980927,8.733797,24.343121,-13.190990,-21.466457,-0.957650,-12.557255,1.0,0.0,0.0,7,0.0,0.0,102.031694,0.0,0.0
1,13.960780,-36.666084,-22.260595,16.980879,8.733829,24.343128,-13.190999,-21.466469,-0.957701,-12.557279,1.0,0.0,0.0,7,0.0,0.0,102.031694,0.0,0.0
2,60.811623,0.946545,-4.255921,20.332489,45.908714,-7.113601,-13.550858,-3.377986,6.305635,-2.023123,1.0,0.0,0.0,5,0.0,0.0,72.057515,0.0,0.0
3,245.147781,29.954426,-0.577181,18.008678,45.957737,14.886771,-29.603376,10.542392,11.490660,12.857011,1.0,0.0,0.0,6,0.0,0.0,86.073165,0.0,0.0
4,245.147781,29.954430,-0.577294,18.008636,45.957726,14.886753,-29.603340,10.542382,11.490674,12.857035,1.0,0.0,0.0,6,0.0,0.0,86.073165,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,254.200806,-89.463516,6.972825,83.248619,9.417171,14.582830,22.372253,14.008273,10.369975,-12.614752,1.0,0.0,0.0,8,0.0,0.0,118.099380,0.0,0.0
152,73.645363,-90.855064,2.386254,67.773544,22.316572,6.024801,30.230356,8.795265,20.386084,-16.042814,1.0,0.0,0.0,9,0.0,0.0,132.115030,0.0,0.0
153,-110.040062,2.599045,-3.462972,-0.363976,3.405136,-12.481925,43.612568,-2.068670,-13.263947,26.234848,1.0,0.0,0.0,10,0.0,0.0,164.086871,0.0,0.0
154,-480.540283,-23.481226,-15.529446,6.733684,3.938678,-16.251133,38.950779,-8.679785,-8.750237,17.551256,1.0,0.0,0.0,13,0.0,0.0,206.133821,0.0,0.0


**Generate salt features**

In [68]:
salt_tens_feat = df["salt_comb_sm"].map(lambda smiles: model(**tokenizer(smiles, return_tensors="pt", padding=True))["logits"][0, 0, :])
salt_tens_feat[0].shape

torch.Size([52000])

In [69]:
salt_tens_feat_stack = torch.stack(salt_tens_feat.tolist()).numpy()
salt_feat_transf = pca.fit_transform(salt_tens_feat_stack)
salt_feat_transf.shape

(156, 10)

In [70]:
salt_dict = {}
for i in range(10):
    feat_name = 'pre_feat_salt_' + str(i+1)
    salt_dict[feat_name] = salt_feat_transf.T[i]

In [71]:
salt_df = pd.DataFrame(salt_dict)
salt_df

,pre_feat_salt_1,pre_feat_salt_2,pre_feat_salt_3,pre_feat_salt_4,pre_feat_salt_5,pre_feat_salt_6,pre_feat_salt_7,pre_feat_salt_8,pre_feat_salt_9,pre_feat_salt_10
0,-221.666412,12.393312,6.670464,-47.994022,62.377254,-7.350951,-0.950416,-0.955368,-7.177319,2.359947
1,-221.666534,12.393236,6.670459,-47.994152,62.377327,-7.350921,-0.950457,-0.955371,-7.177299,2.359975
2,-93.113579,51.149559,-19.272900,-8.102279,-6.797103,18.451069,5.504963,-2.825767,-5.475229,-0.039589
3,-93.113579,51.149727,-19.272722,-8.102569,-6.797097,18.451069,5.504992,-2.825780,-5.475209,-0.039568
4,-93.113579,51.149860,-19.272943,-8.102409,-6.797088,18.451368,5.505139,-2.825732,-5.475221,-0.039548
...,...,...,...,...,...,...,...,...,...,...
151,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151
152,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151
153,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151
154,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151


In [72]:
# salt_df['salt_comb_sm'] = salt_comb
salt_df['salt_1_conc'] = df['salt_1_conc']
salt_df['salt_2_conc'] = df['salt_2_conc']
salt_df['num_heavy_atoms_salt_1'] = num_atom_salt_1
salt_df['num_heavy_atoms_salt_2'] = num_atom_salt_2
salt_df['mw_salt_1'] = mw_salt_1
salt_df['mw_salt_2'] = mw_salt_2
salt_df

,pre_feat_salt_1,pre_feat_salt_2,pre_feat_salt_3,pre_feat_salt_4,pre_feat_salt_5,pre_feat_salt_6,pre_feat_salt_7,pre_feat_salt_8,pre_feat_salt_9,pre_feat_salt_10,salt_1_conc,salt_2_conc,num_heavy_atoms_salt_1,num_heavy_atoms_salt_2,mw_salt_1,mw_salt_2
0,-221.666412,12.393312,6.670464,-47.994022,62.377254,-7.350951,-0.950416,-0.955368,-7.177319,2.359947,1.0,0.0,6,0.0,105.964516,0.0
1,-221.666534,12.393236,6.670459,-47.994152,62.377327,-7.350921,-0.950457,-0.955371,-7.177299,2.359975,1.0,0.0,6,0.0,105.964516,0.0
2,-93.113579,51.149559,-19.272900,-8.102279,-6.797103,18.451069,5.504963,-2.825767,-5.475229,-0.039589,1.0,0.0,8,0.0,195.928020,0.0
3,-93.113579,51.149727,-19.272722,-8.102569,-6.797097,18.451069,5.504992,-2.825780,-5.475209,-0.039568,1.0,0.0,8,0.0,195.928020,0.0
4,-93.113579,51.149860,-19.272943,-8.102409,-6.797088,18.451368,5.505139,-2.825732,-5.475221,-0.039548,1.5,0.0,8,0.0,195.928020,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151,1.0,0.0,10,0.0,186.939685,0.0
152,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151,1.0,0.0,10,0.0,186.939685,0.0
153,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151,1.0,0.0,10,0.0,186.939685,0.0
154,46.835903,-37.557739,-2.260569,-1.691364,-2.132641,0.641856,0.228930,-0.130601,0.112251,0.041151,1.0,0.0,10,0.0,186.939685,0.0


In [73]:
solv_tens_feat[0].size()

torch.Size([52000])

**Generate additive features**

In [74]:
df_ = df.copy()
df_['additive_smiles'][0] = ''
df_['additive_smiles'][0]

''

In [75]:
for i in range(len(df)):
    if type(df['additive_smiles'][i]) == str:
        continue
    else:
        df_['additive_smiles'][i] = ''
df_

,solvent_1_smiles,solvent_2_smiles,solvent_3_smiles,salt_1_smiles,salt_2_smiles,additive_smiles,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,...,pre_feat_add_14,pre_feat_add_15,num_heavy_atoms_add,mw_add,protocol,current_density,coulombic_efficiency,log(1-CE),solv_comb_sm,salt_comb_sm
0,CC1COC(=O)O1,NaN,NaN,[Li+].[O-][Cl+3]([O-])([O-])[O-],NaN,,17.408940,-39.545807,-22.604496,17.312720,...,0.000000e+00,0.000000e+00,0,0.0000,0,2.0,0.800,-1.609438,CC1COC(=O)O1,[Li+].[O-][Cl+3]([O-])([O-])[O-]
1,CC1COC(=O)O1,NaN,NaN,[Li+].[O-][Cl+3]([O-])([O-])[O-],NaN,O=S=O,17.408472,-39.545840,-22.604517,17.312712,...,-1.970000e-09,-7.250000e-10,3,63.9619,1,5.0,0.836,-1.807889,CC1COC(=O)O1,[Li+].[O-][Cl+3]([O-])([O-])[O-]
2,C1CCOC1,NaN,NaN,F[As-](F)(F)(F)(F)F.[Li+],NaN,,64.222480,-1.441071,-4.633460,17.839956,...,0.000000e+00,0.000000e+00,0,0.0000,1,5.0,0.894,-2.244316,C1CCOC1,F[As-](F)(F)(F)(F)F.[Li+]
3,CC1CCCO1,NaN,NaN,F[As-](F)(F)(F)(F)F.[Li+],NaN,,248.554760,27.599460,-1.178202,16.094427,...,0.000000e+00,0.000000e+00,0,0.0000,1,0.9,0.974,-3.649659,CC1CCCO1,F[As-](F)(F)(F)(F)F.[Li+]
4,CC1CCCO1,NaN,NaN,F[As-](F)(F)(F)(F)F.[Li+],NaN,,248.554760,27.599460,-1.178156,16.094429,...,0.000000e+00,0.000000e+00,0,0.0000,1,0.9,0.974,-3.649659,CC1CCCO1,F[As-](F)(F)(F)(F)F.[Li+]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,COCCOC(C)C,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,,257.665560,-92.078210,6.875039,81.282670,...,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.985,-4.199705,COCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]
152,CCOCCOC(C)C,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,,77.102196,-93.227230,2.249297,64.492090,...,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.989,-4.509860,CCOCCOC(C)C,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]
153,CCC[Si](OC)(OC)OC,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,,-106.717125,0.973197,-2.709262,-3.330054,...,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.980,-3.912023,CCC[Si](OC)(OC)OC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]
154,CCCCCC[Si](OC)(OC)OC,NaN,NaN,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+],NaN,,-477.186700,-25.545970,-14.940089,4.034881,...,0.000000e+00,0.000000e+00,0,0.0000,2,0.5,0.857,-1.944911,CCCCCC[Si](OC)(OC)OC,O=S(=O)(F)[N-]S(=O)(=O)F.[Li+]


In [76]:
add_tens_feat = df_["additive_smiles"].map(lambda smiles: model(**tokenizer(smiles, return_tensors="pt", padding=True))["logits"][0, 0, :])
add_tens_feat[0].shape

torch.Size([52000])

In [77]:
add_tens_feat

0      [tensor(-7.6434), tensor(-12.2946), tensor(-8....
1      [tensor(-7.6313), tensor(-10.0034), tensor(-8....
2      [tensor(-7.6434), tensor(-12.2946), tensor(-8....
3      [tensor(-7.6434), tensor(-12.2946), tensor(-8....
4      [tensor(-7.6434), tensor(-12.2946), tensor(-8....
                             ...                        
151    [tensor(-7.6434), tensor(-12.2946), tensor(-8....
152    [tensor(-7.6434), tensor(-12.2946), tensor(-8....
153    [tensor(-7.6434), tensor(-12.2946), tensor(-8....
154    [tensor(-7.6434), tensor(-12.2946), tensor(-8....
155    [tensor(-7.6434), tensor(-12.2946), tensor(-8....
Name: additive_smiles, Length: 156, dtype: object

In [78]:
add_tens_feat_stack = torch.stack(add_tens_feat.tolist()).numpy()
add_tens_feat_stack

array([[ -7.643441 , -12.294619 ,  -8.164743 , ..., -12.907577 ,
        -12.294504 , -12.7542515],
       [ -7.631347 , -10.003353 ,  -8.125792 , ..., -10.832726 ,
         -9.729509 ,  -9.974509 ],
       [ -7.643441 , -12.294619 ,  -8.164743 , ..., -12.907577 ,
        -12.294504 , -12.7542515],
       ...,
       [ -7.643441 , -12.294619 ,  -8.164743 , ..., -12.907577 ,
        -12.294504 , -12.7542515],
       [ -7.643441 , -12.294619 ,  -8.164743 , ..., -12.907577 ,
        -12.294504 , -12.7542515],
       [ -7.643441 , -12.294619 ,  -8.164743 , ..., -12.907577 ,
        -12.294504 , -12.7542515]], dtype=float32)

In [79]:
add_feat_transf = pca.fit_transform(add_tens_feat_stack)
add_feat_transf.shape

(156, 10)

In [80]:
add_dict = {}
for i in range(10):
    feat_name = 'pre_feat_add_' + str(i+1)
    add_dict[feat_name] = add_feat_transf.T[i]

In [81]:
add_df = pd.DataFrame(add_dict)
add_df

,pre_feat_add_1,pre_feat_add_2,pre_feat_add_3,pre_feat_add_4,pre_feat_add_5,pre_feat_add_6,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10
0,-90.638054,-7.459128,0.380419,-0.838915,-0.130742,-0.516739,-0.095595,-0.167923,-0.039421,0.002880
1,412.144104,-53.276897,-22.963917,7.520813,21.922449,23.375114,0.481968,13.548994,52.096531,0.002881
2,-90.638023,-7.459107,0.380672,-0.838962,-0.130806,-0.516744,-0.095610,-0.167903,-0.039404,0.002921
3,-90.638023,-7.459123,0.380390,-0.838924,-0.130624,-0.516735,-0.095616,-0.167928,-0.039432,0.002854
4,-90.638023,-7.459123,0.380368,-0.838932,-0.130724,-0.516762,-0.095605,-0.167958,-0.039389,0.002909
...,...,...,...,...,...,...,...,...,...,...
151,-90.638023,-7.459124,0.380373,-0.838922,-0.130731,-0.516760,-0.095591,-0.167941,-0.039426,0.002881
152,-90.638023,-7.459124,0.380373,-0.838922,-0.130731,-0.516760,-0.095591,-0.167941,-0.039426,0.002881
153,-90.638023,-7.459124,0.380373,-0.838922,-0.130731,-0.516760,-0.095591,-0.167941,-0.039426,0.002881
154,-90.638023,-7.459124,0.380373,-0.838922,-0.130731,-0.516760,-0.095591,-0.167941,-0.039426,0.002881


In [82]:
## replace non-additive entries with 0
cols_add = add_df.columns
num_atom_add = []
mw_add = []
for i in range(len(df)):
    if type(df['additive_smiles'][i]) == str:
        mol = Chem.MolFromSmiles(df['additive_smiles'][i])
        num_atom_add.append(MolDescriptors.CalcNumHeavyAtoms(mol))
        mw_add.append(MolDescriptors.CalcExactMolWt(mol))
        continue
    else:
        num_atom_add.append(0.0)
        mw_add.append(0.0)
        for j in range(10):
            add_df[cols_add[j]][i] = 0.0
add_df

,pre_feat_add_1,pre_feat_add_2,pre_feat_add_3,pre_feat_add_4,pre_feat_add_5,pre_feat_add_6,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,412.144104,-53.276897,-22.963917,7.520813,21.922449,23.375114,0.481968,13.548994,52.096531,0.002881
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
151,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
153,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [83]:
mw_add

[0.0,
 63.961900240000006,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 131.88809666999998,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 69.00382241000001,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 86.00039392400001,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 86.00039392400001,
 106.00662217600001,
 106.00662217600001,
 0.0,
 277.86963285,
 0.0,
 106.00662217600001,
 0.0,
 106.00662217600001,
 69.00382241000001,
 0.0,
 0.0,
 86.00039392400001,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 86.00039392400001,
 0.0,
 0.0,
 0.0,
 69.00382241000001,
 0.0,
 0.0,
 88.016043988,
 88.016043988,
 88.016043988,
 88.016043988,
 88.016043988,
 88.016043988,
 88.016043988,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 69.00382241000001,
 106.00662217600001,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 123.98302960800001,
 123.98302960800001,
 0.0,
 0.0,
 0.0,
 0.

In [84]:
add_df['num_heavy_atoms_add'] = num_atom_add
add_df['mw_add'] = mw_add
add_df

,pre_feat_add_1,pre_feat_add_2,pre_feat_add_3,pre_feat_add_4,pre_feat_add_5,pre_feat_add_6,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10,num_heavy_atoms_add,mw_add
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
1,412.144104,-53.276897,-22.963917,7.520813,21.922449,23.375114,0.481968,13.548994,52.096531,0.002881,3.0,63.9619
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...
151,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
152,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
153,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
154,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000


In [85]:
final_df = pd.concat([solv_df, salt_df, add_df], axis=1)
final_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10,...,pre_feat_add_3,pre_feat_add_4,pre_feat_add_5,pre_feat_add_6,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10,num_heavy_atoms_add,mw_add
0,13.960964,-36.666103,-22.260607,16.980927,8.733797,24.343121,-13.190990,-21.466457,-0.957650,-12.557255,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
1,13.960780,-36.666084,-22.260595,16.980879,8.733829,24.343128,-13.190999,-21.466469,-0.957701,-12.557279,...,-22.963917,7.520813,21.922449,23.375114,0.481968,13.548994,52.096531,0.002881,3.0,63.9619
2,60.811623,0.946545,-4.255921,20.332489,45.908714,-7.113601,-13.550858,-3.377986,6.305635,-2.023123,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
3,245.147781,29.954426,-0.577181,18.008678,45.957737,14.886771,-29.603376,10.542392,11.490660,12.857011,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
4,245.147781,29.954430,-0.577294,18.008636,45.957726,14.886753,-29.603340,10.542382,11.490674,12.857035,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,254.200806,-89.463516,6.972825,83.248619,9.417171,14.582830,22.372253,14.008273,10.369975,-12.614752,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
152,73.645363,-90.855064,2.386254,67.773544,22.316572,6.024801,30.230356,8.795265,20.386084,-16.042814,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
153,-110.040062,2.599045,-3.462972,-0.363976,3.405136,-12.481925,43.612568,-2.068670,-13.263947,26.234848,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000
154,-480.540283,-23.481226,-15.529446,6.733684,3.938678,-16.251133,38.950779,-8.679785,-8.750237,17.551256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000


In [86]:
final_df['protocol'] = df['protocol']
final_df['current_density'] = df['current_density']
final_df['coulombic_efficiency'] = df['coulombic_efficiency']
final_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10,...,pre_feat_add_6,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10,num_heavy_atoms_add,mw_add,protocol,current_density,coulombic_efficiency
0,13.960964,-36.666103,-22.260607,16.980927,8.733797,24.343121,-13.190990,-21.466457,-0.957650,-12.557255,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0,2.0,0.800
1,13.960780,-36.666084,-22.260595,16.980879,8.733829,24.343128,-13.190999,-21.466469,-0.957701,-12.557279,...,23.375114,0.481968,13.548994,52.096531,0.002881,3.0,63.9619,1,5.0,0.836
2,60.811623,0.946545,-4.255921,20.332489,45.908714,-7.113601,-13.550858,-3.377986,6.305635,-2.023123,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,5.0,0.894
3,245.147781,29.954426,-0.577181,18.008678,45.957737,14.886771,-29.603376,10.542392,11.490660,12.857011,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,0.9,0.974
4,245.147781,29.954430,-0.577294,18.008636,45.957726,14.886753,-29.603340,10.542382,11.490674,12.857035,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,0.9,0.974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,254.200806,-89.463516,6.972825,83.248619,9.417171,14.582830,22.372253,14.008273,10.369975,-12.614752,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.985
152,73.645363,-90.855064,2.386254,67.773544,22.316572,6.024801,30.230356,8.795265,20.386084,-16.042814,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.989
153,-110.040062,2.599045,-3.462972,-0.363976,3.405136,-12.481925,43.612568,-2.068670,-13.263947,26.234848,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.980
154,-480.540283,-23.481226,-15.529446,6.733684,3.938678,-16.251133,38.950779,-8.679785,-8.750237,17.551256,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.857


In [87]:
log_ce = np.log(1-df['coulombic_efficiency'])
log_ce

0     -1.609438
1     -1.807889
2     -2.244316
3     -3.649659
4     -3.649659
         ...   
151   -4.199705
152   -4.509860
153   -3.912023
154   -1.944911
155   -3.324236
Name: coulombic_efficiency, Length: 156, dtype: float64

In [88]:
final_df['log(1-CE)'] = log_ce
final_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10,...,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10,num_heavy_atoms_add,mw_add,protocol,current_density,coulombic_efficiency,log(1-CE)
0,13.960964,-36.666103,-22.260607,16.980927,8.733797,24.343121,-13.190990,-21.466457,-0.957650,-12.557255,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0,2.0,0.800,-1.609438
1,13.960780,-36.666084,-22.260595,16.980879,8.733829,24.343128,-13.190999,-21.466469,-0.957701,-12.557279,...,0.481968,13.548994,52.096531,0.002881,3.0,63.9619,1,5.0,0.836,-1.807889
2,60.811623,0.946545,-4.255921,20.332489,45.908714,-7.113601,-13.550858,-3.377986,6.305635,-2.023123,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,5.0,0.894,-2.244316
3,245.147781,29.954426,-0.577181,18.008678,45.957737,14.886771,-29.603376,10.542392,11.490660,12.857011,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,0.9,0.974,-3.649659
4,245.147781,29.954430,-0.577294,18.008636,45.957726,14.886753,-29.603340,10.542382,11.490674,12.857035,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,0.9,0.974,-3.649659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,254.200806,-89.463516,6.972825,83.248619,9.417171,14.582830,22.372253,14.008273,10.369975,-12.614752,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.985,-4.199705
152,73.645363,-90.855064,2.386254,67.773544,22.316572,6.024801,30.230356,8.795265,20.386084,-16.042814,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.989,-4.509860
153,-110.040062,2.599045,-3.462972,-0.363976,3.405136,-12.481925,43.612568,-2.068670,-13.263947,26.234848,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.980,-3.912023
154,-480.540283,-23.481226,-15.529446,6.733684,3.938678,-16.251133,38.950779,-8.679785,-8.750237,17.551256,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.857,-1.944911


In [89]:
all_df = final_df.iloc[:142,:]
all_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10,...,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10,num_heavy_atoms_add,mw_add,protocol,current_density,coulombic_efficiency,log(1-CE)
0,13.960964,-36.666103,-22.260607,16.980927,8.733797,24.343121,-13.190990,-21.466457,-0.957650,-12.557255,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,0,2.0,0.80000,-1.609438
1,13.960780,-36.666084,-22.260595,16.980879,8.733829,24.343128,-13.190999,-21.466469,-0.957701,-12.557279,...,0.481968,13.548994,52.096531,0.002881,3.0,63.9619,1,5.0,0.83600,-1.807889
2,60.811623,0.946545,-4.255921,20.332489,45.908714,-7.113601,-13.550858,-3.377986,6.305635,-2.023123,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,5.0,0.89400,-2.244316
3,245.147781,29.954426,-0.577181,18.008678,45.957737,14.886771,-29.603376,10.542392,11.490660,12.857011,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,0.9,0.97400,-3.649659
4,245.147781,29.954430,-0.577294,18.008636,45.957726,14.886753,-29.603340,10.542382,11.490674,12.857035,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,1,0.9,0.97400,-3.649659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,5.402024,69.067390,-2.202446,-12.765357,31.526262,10.880734,18.148718,6.605233,-6.500397,11.288347,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.99759,-6.028129
138,488.717041,59.473896,17.828417,-12.404927,7.683675,46.233986,-3.479155,10.291774,12.132977,9.491340,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.98200,-4.017384
139,-157.245941,-93.020432,51.873348,-13.462182,-8.900540,18.964680,-14.528912,-11.747053,21.902327,22.804018,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,0.5,0.98800,-4.422849
140,-302.425690,-50.619164,7.512139,-11.243821,0.393674,18.516932,3.606309,-2.471229,-19.082096,2.865817,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0000,2,1.0,0.98700,-4.342806


In [90]:
ood_df = final_df.iloc[142:,:]
ood_df

,pre_feat_solv_1,pre_feat_solv_2,pre_feat_solv_3,pre_feat_solv_4,pre_feat_solv_5,pre_feat_solv_6,pre_feat_solv_7,pre_feat_solv_8,pre_feat_solv_9,pre_feat_solv_10,...,pre_feat_add_7,pre_feat_add_8,pre_feat_add_9,pre_feat_add_10,num_heavy_atoms_add,mw_add,protocol,current_density,coulombic_efficiency,log(1-CE)
142,568.741333,57.725716,52.314514,-26.843203,13.405765,8.493670,3.161298,25.868860,-8.571390,-16.418213,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.991411,-4.757323
143,314.174316,136.484940,26.495731,-17.763567,15.258575,6.446876,6.723159,-17.591743,4.628298,-17.381075,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.874748,-2.077424
144,696.377747,107.865707,38.945183,-24.380642,-12.844133,-15.364827,-1.055941,-16.993942,-21.821970,10.203899,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.974297,-3.661161
145,-495.069214,19.423439,-9.941760,11.514219,8.910452,-9.091766,26.418667,6.267409,-18.140549,16.178228,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.984961,-4.197091
146,155.699402,-110.856628,5.280406,77.235680,9.402537,8.391991,25.351292,9.018285,14.835892,-20.774914,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.974294,-3.661018
147,67.646698,155.913681,11.699912,-25.080843,-0.243393,-24.124809,-8.694117,13.481190,-5.478823,33.760818,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.550291,-0.799155
148,696.377747,107.865707,38.945183,-24.380642,-12.844133,-15.364827,-1.055941,-16.993942,-21.821970,10.203899,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.1,0.966902,-3.408275
149,307.999420,-51.896534,12.827153,70.621666,17.702200,16.891184,26.094503,13.102829,14.195604,-10.032532,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.982281,-4.033095
150,0.473899,113.481331,17.095774,19.033155,-10.999088,-10.033943,5.146527,-14.117541,1.496447,-4.443938,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.983000,-4.074542
151,254.200806,-89.463516,6.972825,83.248619,9.417171,14.582830,22.372253,14.008273,10.369975,-12.614752,...,0.0,0.0,0.0,0.0,0.0,0.0,2,0.5,0.985000,-4.199705


In [51]:
final_df.to_csv('tot_CE_ED-ood_pre_feat.csv', index=False)
all_df.to_csv('all_CE_ED_pre_feat.csv', index=False)
ood_df.to_csv('CE_ood_pre_feat.csv', index=False)